---
# Jupyter-Notebook-Tutorial:   
#    Anpassung von Modellen an Daten mit *kafe2* 

                                                Johannes Gäßler, März 2021
                                                Günter Quast, April 2020
---
## Grundsätzliches zu Jupyter Notebooks

Diese Datei vom Typ `.ipynb` enthält ein Tutorial als `Jupyter notebook`.
*Jupyter* bietet eine Browser-Schnittstelle mit einer (einfachen) Entwicklungsumgebung für
*Python*-Code und erklärende Texte im intuitiven *Markdown*-Format.
Die Eingabe von Formeln im *LaTeX*-Format wird ebenfalls unterstützt.

Eine Zusammenstellung der wichtigsten Befehle zur Verwendung von *Jupyter* als Arbeitsumgebung
findet sich im Notebook
[*JupyterCheatsheet.ipynb*](https://git.scc.kit.edu/yh5078/datenanalyse/-/blob/master/jupyter/JupyterCheatsheet.ipynb).
Grundlagen zur statistischen Datenauswertung finden sich in den Notebooks 
[*IntroStatistik.ipynb*](https://git.scc.kit.edu/yh5078/datenanalyse/-/blob/master/jupyter/IntroStatistik.ipynb)
und
[*Fehlerrechnung.ipynb*](https://git.scc.kit.edu/yh5078/datenanalyse/-/blob/master/jupyter/Fehlerrechnung.ipynb).

In *Jupyter* werden Code und Text in jeweils einzelne Zellen eingegeben. 
Aktive Zellen werden durch einen blauen Balken am Rand angezeigt.
Sie können sich in zwei Zuständen befinden: im Edit-Mode ist das Eingabefeld weiß, im
Command-Mode ist es ausgegraut.
Durch Klicken in den Randbereich wird der Command-Mode gewählt, ein Klick in das Textfeld einer
Code-Zelle schaltet in den Edit-Mode.
Die Taste `esc` kann ebenfalls verwendet werden, um den Edit-Mode zu verlassen.

Die Eingabe von `a` im Command-Mode erzeugt eine neue leere Zelle oberhalb der aktiven Zelle, `b`
eine unterhalb. Eingabe von `dd` löscht die betreffende Zelle.

Zellen können entweder den Typ `Markdown` oder `Code` haben.
Die Eingabe von `m` im Command-Mode setzt den Typ Markdown, Eingabe von `y` wählt den Typ Code.

Prozessiert - also Text gesetzt oder Code ausgeführt - wird der Zelleninhalt durch Eingabe von
`shift+return`, oder auch `alt+return` wenn zusätzlich eine neue, leere Zelle erzeugt werden soll.

Die hier genannten Einstellungen sowie das Einfügen, Löschen oder Ausführen von Zellen sind
auch über das PullDown-Menü am oberen Rand verfügbar.

---

# Übersicht: *kafe*2
***


*kafe2* ist ist eine erweiterte Version des seit 2012 entwickelten Pakets *kafe* zur Anpassung
von Modellfunktionen an Daten.

Unterstützt werden verschiedene Datentypen wie einfache indizierte Daten, zweidimensionale
Datenpunkte (eine Größe *x* und eine abhängige Größe *y*) sowie Häufigkeitsverteilungen
(Histogramme).
Unsicherheiten sowohl der abhängigen als auch der unabhängigen Größen und gegebenenfalls deren
Korrelationen werden unterstützt.
Dazu wird aus verschiedenen Arten von spezifizierten Unsicherheiten die globale Kovarianzmatrix
erstellt und in der Anpassung berücksichtigt.
Im Vergleich zu vielen anderen Anpassungswerkzeugen ist diese Möglichkeit ein
Alleinstellungsmerkmal von *kafe(2)*.

Unterstützt wird auch die gleichzeitige Anpassung mehrerer Modelle mit jeweils eigenen und
zusätzlich allen oder mehreren Modellen zugehörigen Parametern an verschiedene Datensätze
("Multi-Fit").

Zur Minimierung des Abstandsmaßes zwischen Daten und Modellfunktion(en) werden numerische
Verfahren angewandt, die aus der quelloffenen, *Python*-basierten Softwareumgebung *SciPy* oder
dem am CERN entwickelten Paket *MINUIT* stammen.
Das jeweils minimierte Abstandsmaß (oder auch die "Kostenfunktion") entspricht dem mit einem
Faktor Zwei multiplizierten negativen natürlichen Logarithmus der Likelihood-Funktion
$-2\,\ln{\cal L}$ der Daten für das gegebene Modell.
Für Gauß-förmige Unsicherheiten der Datenpunkte entspricht dies der Methode der kleinsten
Quadrate (auch "$\chi^2$-Methode").
Andere, auf dem Likelihood-Prinzip beruhende Kostenfunktionen für die Anpassung von
Wahrscheinlichkeitsdichten an Histogramme oder an indizierte Daten sind ebenfalls verfügbar.

Zur Bestimmung der Unsicherheiten auf die Parameter des angepassten Modells wird die Methode der
Profil-Likelihood bereit gestellt, mit deren Hilfe Konfidenzintervalle für die einzelnen
Parameter sowie zweidimensionale Konfidenz-Konturen für Paare von Parametern bestimmt werden
können.

*kafe2* enthält eine stand-alone Anwendung *kafe2go*, die Anpassungen ohne die Erstellung von
eigenem Code ermöglicht;
Daten, Modellfunktion und Optionen werden dazu in einer Konfigurationsdatei im *YAML*-Format angegeben.
Mit dem folgenden Konsolenbefehl kann daraus ein Fit erstellt werden:

`kafe2go <name>.yaml`

*kafe2* kann aber ebenfalls und sehr viel flexibler über ein *Python*-Interface verwendet werden.
Eine Einführung in die Möglichkeiten gibt dieses Tutorial.
Generell ist die Vorgehensweise folgende:

  - Definition und Initialisierung eines Daten-Containers für die jeweilige Anpassung  
    (Klassen IndexedContainer, XYContainer, HistContainer, UnbinnedContainer).
  - Erzeugung eines geeigneten Objekts zur Durchführung der Anpassung, die den Datencontainer mit
    einem Modell verbindet  (die allgemeine Klasse *Fit* bzw. spezialisierte Klassen
    *IndexedFit*, *XYFit*, *HistFit*, *UnbinnedFit*).
  - Durchführung der Anpassung mittels $<$Fit_Objekt$>$.do_fit() und Ausgabe der Ergebnisse auf der
    Konsole mittels $<$Fit_Objekt$>$.report() oder durch direkten Zugriff auf die
    Ergebnis-Variablen des Fit-Objekts.
  - Gegebenenfalls Erzeugung und Anzeige von Ergebnisgrafiken mit der generischen Klasse *Plot*.
    
**Die folgenden Beispiele zeigen die konkrete Vorgehensweise.**

#### Allgemeine Einstellungen und nützliche Pakete

In [ ]:
from __future__ import division, print_function  # Python2-Kompatibilität
import sys, os

# Zeilen mit % oder %% am Anfang sind sogenannte "magische Kommandos",
# die den Zellentyp oder Optionen für die Anzeige von Grafiken festlegen.
%matplotlib inline

#### Imports und Voreinstellungen für *kafe2*:

In [ ]:
from kafe2 import config, XYContainer, Fit, Plot

import numpy as np, matplotlib.pyplot as plt

# set better default figure size for kafe2
# plt.rcParams['figure.figsize']=[12., 5.] 
#        !!!  must be done after importing kafe2 (will else be overwritten)


***
## 1. Einfaches Beispiel zur Funktionsanpassung mit *kafe2*
***

Der folgende Code illustriert die Anpassung von Funktionen mit dem Anpassungswerkzeug *kafe2*:
```
# Create an XYContainer object to hold the xy data for the fit:
xy_data = XYContainer(x_data=[1.0, 2.0, 3.0, 4.0],
                      y_data=[2.3, 4.2, 7.5, 9.4])
# x_data and y_data are combined depending on their order.
# The above translates to the points (1.0, 2.3), (2.0, 4.2), (3.0, 7.5), and (4.0, 9.4).

# Important: Specify uncertainties for the data:
xy_data.add_error(axis='x', err_val=0.1)
xy_data.add_error(axis='y', err_val=0.4)

xy_data.label = 'Data'  # How the data is called in plots

# Create an XYFit object from the xy data container.
# By default, a linear function f=a*x+b will be used as the model function.
line_fit = Fit(data=xy_data)

# Perform the fit: Find values for a and b that minimize the
#     difference between the model function and the data.
line_fit.do_fit()  # This will throw a warning if no errors were specified.

# Optional: Print out a report on the fit results on the console.
line_fit.report()

# Optional: Create a plot of the fit results using Plot.
plot = Plot(fit_objects=line_fit)  # Create a kafe2 plot object.
plot.x_label = 'x'  # Set x axis label.
plot.y_label = 'y'  # Set y axis label.
plot.plot()  # Do the plot.

plot.save()  # Saves the plot to file 'fit.png' .
# plot.save('my_fit.pdf')  # Saves the plot to a different file / with a different file extension.

# Show the fit result.
plot.show()  # Just a convenience wrapper for matplotlib.pyplot.show() .
# NOTE: Calling matplotlib.pyplot.show() closes all figures by default so call this AFTER saving.
```

Fügen Sie den Code in die leere Zelle unten ein und führen Sie sie durch Eingabe von `shift+return`
aus.

In [ ]:
# einfaches Beispiel: Geradenanpassung mit kafe2
# -> Code hier einfügen 



### 1.1 Korrelierte Unsicherheiten
***

Zur Illustration der Möglichkeiten zur Behandlung von Unsicherheiten fügen wir eine weitere
korrelierte Unsicherheit der abhängigen Größen *y* ein:
```
xy_data.add_error(axis='y', err_val=0.3, correlation=1.)
```

Fügen Sie diese Zeile in den obigen Code ein.
Die Ausgabe können Sie etwas übersichtlicher gestalten, indem Sie auf die Ausgabe der Daten und
des Modells verzichten.
Modifizieren Sie dazu die Parameter der *report()*-Methode:
```
line_fit.report(show_data=False, show_model=False)
```
Wiederholen Sie nun die Anpassung.

Wie erwartet wirkt sich eine solche allen Datenpunkten gemeinsame Unsicherheit nicht auf die
Steigung der Geraden, sondern nur auf den Parameter *b* aus.
Dessen Unsicherheit wird nun größer - entsprechend der Wurzel aus der
quadratischen Summe der Unsicherheiten von +/-0.58 aus der
ursprünglichen Anpassung und der zusätzlichen korrelierten Unsicherheit von +/-0.40.

In [ ]:
##Code aus dem vorigen Beispiel kopieren und ergänzen
# ->


***
## 2. Vergleich von zwei verschiedenen Modellen 
***

Einfache, in den Parametern lineare Modelle reichen in der Praxis nicht aus. 
Das folgende Beispiel zeigt die Anpassung eines linearen und eines exponentiellen Modells an die
gleichen Daten.

Um eine Modellfunktion für *kafe2* zu definieren, genügt es, eine *Python*-Funktion
zu schreiben.
Wichtig:
das erste Argument der *Python*-Funktion wird als unabhängige Variable interpretiert.
Das erste Argument wird während der Anpassung also nicht modifiziert und es ist die Größe,
die vom Fit als x-Achse interpretiert wird.

Definition von zwei Modellfunktionen:
```
# Our first model is a simple linear function:
def linear_model(x, a, b):
    return a * x + b


# Our second model is a simple exponential function.
# The kwargs in the function header specify parameter defaults.
def exponential_model(x, A0=1., x0=5.):
    return A0 * np.exp(x/x0)
```

Hier die Definition der Daten als *kafe2* `XYContainer`:
```
# The data for this exercise:
x = [19.8, 3.0, 5.1, 16.1, 8.2, 11.7, 6.2, 10.1]
y = [23.2, 3.2, 4.5, 19.9, 7.1, 12.5, 4.5, 7.2]
data2 = XYContainer(x_data=x, y_data=y)
data2.add_error(axis='x', err_val=0.3)
data2.add_error(axis='y', err_val=0.15, relative=True)
```

Damit die Daten in der grafischen Ausgabe später klar gekennzeichnet werden,
werden noch Namen für den Datensatz und die Achsenbeschriftungen gesetzt:
```
data2.label = 'Datenpunkte'
data2.axis_labels=['x-Wert', 'y-Wert']
```

Dann wird je ein Fit-Objekt mit den beiden Modell-Funktionen und jeweils
den gleichen Daten erzeugt:
```
# Create 2 Fit objects with the same data but with different model functions:
linear_fit = Fit(data2, model_function=linear_model)
exponential_fit = Fit(data2, model_function=exponential_model)
```

Damit alles schöner aussieht, definieren wir noch LaTeX-Ausdrücke für die Funktionen,
die Namen der Parameter und die Legende in der grafischen Ausgabe:
```
# Optional: Assign LaTeX strings to parameters and model functions.
linear_fit.assign_parameter_latex_names(a='a', b='b')
linear_fit.assign_model_function_latex_expression("{a}{x} + {b}")
linear_fit.model_label = 'lineares Modell'
exponential_fit.assign_parameter_latex_names(A0='A_0', x0='x_0')
exponential_fit.assign_model_function_latex_expression("{A0} e^{{{x}/{x0}}}")
exponential_fit.model_label = 'exponentielles Modell'
```

Der Code zur Ausführung der Anpassungen sieht wie folgt aus:
```
# Perform the fits:
linear_fit.do_fit()
exponential_fit.do_fit()

# Optional: Print out a report on the result of each fit.
linear_fit.report()
exponential_fit.report()

# Optional: Create a plot of the fit results using Plot.
p = Plot(fit_objects=[linear_fit, exponential_fit], separate_figures=False)
p.plot(fit_info=True)

# Show the fit results:
plt.show()
```

Fügen Sie den Code in die leere Zelle unten ein und führen Sie sie durch Eingabe von
`shift+return` aus.

In [ ]:
# Vergleich von zwei Modellen mit kafe2
# -> code hier einfügen 



### 2.1 Hypothesentest zur Bewertung der Modelle
***

Die grafische Ausgabe lässt nicht klar erkennen, welches der Modelle akzeptabel ist. 
Dazu kann ein Hypothesentest ausgeführt werden, der die sogenannte $\chi^2$-Wahrscheinlichkeit
angibt - also die Wahrscheinlichkeit dafür, einen schlechteren Wert von
$\chi^2$ am Minimum zu erhalten als den beobachteten.
Ein höherer Wert entspricht einem besseren Fit.
 
Berechnet wird er aus der kumulativen Verteilungsdichte der Chi2-Funktion:
```
from scipy import stats

def chi2prob(chi2, ndf):
  """ chi2-probability
 
    Args:
      * chi2: chi2 value
      * ndf: number of degrees of freedom

    Returns:
      * float: chi2 probability
  """

  return 1.- stats.chi2.cdf(chi2, ndf)
```

Geben Sie den Code für die $\chi^2$-Wahrscheinlichkeit in die leere Zelle unten ein und
überprüfen Sie die beiden Ergebnisse, die Sie oben erhalten haben.

**Hinweis**: Sie können die Werte für $\chi^2$ und die Zahl der Freiheitsgrade entweder
aus der Ausgabe der vorigen Zelle abtippen oder Sie können die Werte über die Properties `goodness_of_fit` und `ndf` der Fits beziehen.
Das Property `cost_function_value` ist nicht geeignet, da es neben $\chi^2$
auch Korrekturterme enthält.

In [ ]:
# Überprüfung der Qualität der Anpassungen
# -> code hier eingeben


Bei der Anwendung von *kafe2* außerhalb dieses Notebooks empfiehlt es sich, die
$\chi^2$-Wahrscheinlichkeit einfach über das entsprechende Property der Fit-Objekte
zu beziehen. 
Für die obigen Fits könnte dies z.B. über `chi2_prob = linear_fit.chi2_probability` erfolgen.

### 2.2 Untersuchung des nichtlinearen Modells
***

Bei Modellfunktionen, die nichtlinear in den Parametern sind, ist die $\chi^2$-Verteilung
um das Minimum nur näherungsweise eine Parabel, bisweilen weicht sie sogar stark davon ab.
Ob die Abweichungen vernachlässigbar klein sind, kann mit Hilfe der Profil-Likelihood und
durch die Darstellung von Konfidenz-Konturen überprüft werden.
```
from kafe2 import ContoursProfiler

# Create contour plot and profiles for the exponential fit:
cpf = ContoursProfiler(exponential_fit)
cpf.plot_profiles_contours_matrix(show_grid_for='contours')
plt.show()
```

Geben Sie das Code-Beispiel in der Zelle unten ein und überprüfen Sie damit die Anpassung des
exponentiellen Modells.

In [ ]:
# Überprüfung der nichtlinearen Anpassung
# -> code hier eingeben


### 2.3 Asymmetrische Parameterunsicherheiten
***

Wenn die Abweichungen groß sind, müssen asymmetrische Unsicherheiten angegeben werden.
Dazu wird in der *report*-Funktion der Fit-Klasse die Option *asymmetric_parameter_errors=True*
gesetzt.
```
exponential_fit.report(asymmetric_parameter_errors=True)
```

Es empfiehlt sich, in solchen Fällen mit stark asymmetrischen Parameterunsicherheiten die
Konturen zu dokumentieren, wenn mehr als ein Parameter von Interesse ist.

In [ ]:
# hier Code zur Ausgabe asymmetrischer Parameter eingeben
# ->


**Hinweis**: Sie können über `Plot.plot(asymmetric_parameter_errors=True)` auch in der
grafischen Ausgabe die asymmetrischen Parameterfehler einblenden.

### 2.4 Beeinflussung der grafischen Ausgabe
***

Die grafische Ausgabe war noch nicht in allen Belangen optimal. 
Es wurden in der Legende z.B. zwei Datensätze angegeben, obwohl es für beide Modelle nur den
gleichen gab.
Außerdem lassen sich Markereigenschaften und Farben anpassen.

Zur Beeinflussung der Grafik enhält *kafe2* eine Methode `Plot.customize()`, mit deren Hilfe für
die verschiedenen Grafikelemente (*plot_types*: 'data', 'model_line', 'model_error_band',
'ratio', 'ratio_error_band') Werte für *matplotlib*-Parameter angegeben werden können.

Die für einen *plot_type* relevanten Parameter und deren momentane Werte lassen sich über eine
Funktion der *Plot*-Klasse anzeigen:
```
p.get_keywords('model_error_band')
```

Die verwendeten Namen für Objekte und mögliche Werte entstprechen den Bezeichnungen in der
Konfigurationsdatei *matplotlibrc* für *matplotlib*.

Zur Änderung des Namens für den Datensatz und die Unterdrückung der zweiten Ausgabe dient
folgender Aufruf:
```
p.customize('data', 'label', ["test data", None])
```
Das erste Argument spezifiziert den Teil des Plots, der modifiziert werden soll.
Das zweite Argument spezifiziert welches Keyword gesetzt werden soll.
Das dritte Argument ist eine Liste mit Werten, die jeweils für das Keyword bei den vom
Plot-Objekt verwalteten Fit-Objekten gesetzt werden sollen.

Alternativ kann für das dritte Argument auch eine Liste an Tupeln aus Fit-Indices und
Werten übergeben werden:
```
p.customize('data', 'label', [(0, "test data"), (1, None)])
```
Mit dieser Syntax genügt es, nur für einen Teil der Fits Werte anzugeben.

Auch Marker-Typ, Größe und Farbe des Markers und der Fehlerbalken lassen sich anpassen:
```
# data
p.customize('data', 'marker', ['o', 'o'])
p.customize('data', 'markersize', [5, 5])
p.customize('data', 'color', [(0, 'blue'), (1,'blue')]) # note: although 2nd label is suppressed
p.customize('data', 'ecolor', [(0, 'blue'), (1, 'blue')]) # note: although 2nd label is suppressed
```

Ebenso können die entsprechenden Werte für die Modellfunktion angepasst werden:
```
# model
p.customize('model_line', 'color', ['orange', 'lightgreen'])
p.customize('model_error_band', 'label', [(0, r'$\pm 1 \sigma$'), (1, r'$\pm 1 \sigma$')])
p.customize('model_error_band', 'color', [(0, 'orange')])
p.customize('model_error_band', 'color', [(1, 'lightgreen')])
```

Es ist auch möglich, Parameter über die *matplotlib*-Funktionen zu verändern. 
Um die Größe der Achsenbeschriftungen zu ändern, verwendet man z.B. folgende Aufrufe:
```
# Größe der Achsenbeschriftungen
import matplotlib as mpl
mpl.rc('axes', labelsize=20, titlesize=25)
```
Achtung: der obige Aufruf führt zu einer globalen Änderung der *matplotlib*-Parameter.
Plots außerhalb von *kafe2* werden also auch beeinflusst.

Natürlich muss nach diesen Änderungen die Ausgabegrafik neu erzeugt und angezeigt werden:
```
p.plot()
plt.show()
```

In [ ]:
# code zum Testen hier eingeben:
# --> 


### 2.5 Ausgabe der Anpassungsergebnisse als Variable 
***

In vielen Anwendungen ist es nötig, die Ausgabe einer Anpassung im Programmcode weiter zu verwenden.
Dazu dient die *kafe2*-Funktion *Fit.get_result_dict()*, die ein *Python*-Dictionary mit den
Fit-Ergebnissen zurückgibt.

Zum Beispiel:
```
result_0 = <Fit_object>.get_result_dict()
```

Eine formatierte Ausgabe erhält man mit folgender Zeile:
```
print("\n".join("{}\t{}".format(k, v) for k, v in result_0.items()))
```
Die im Dictionary enthaltenen Werte können auch über Properties des Fit-Objektes
bezogen werden.
Achtung: die Namen der Properties sind zum Teil etwas unterschiedlich,
mehr dazu weiter unten.

In [ ]:
# Ausgabe hier testen 
# --> 


### 2.6 Nicht-Linearität durch Fehler in *x* - Richtung
***

Wenn die Fehler in x-Richtung vergrößert werden, wird auch die Anpassung einer Geraden
zu einem nicht-linearen Problem.
Zur Illustration wiederholen wir die gleiche Anpassung wie oben mit vergrößerten
Unsicherheiten auf die x-Werte:
```
# The data for this exercise:
data3 = XYContainer(x_data=x, y_data=y)
data3.add_error(axis='x', err_val=1.0)  # Was 0.3 before.
data3.add_error(axis='y', err_val=0.15, relative=True)

# Create 2 Fit objects with the same data but with different model functions:
linear_fit2 = Fit(data3, model_function=linear_model)

# Optional: Assign LaTeX strings to parameters and model functions.
linear_fit2.assign_parameter_latex_names(x='x', a='a', b='b')
linear_fit2.assign_model_function_latex_expression("{a}{x} + {b}")

# Perform the fits.
linear_fit2.do_fit()

# Optional: Print out a report on the result of each fit.
#linear_fit2.report()

# Optional: Create a plot of the fit results using Plot.
p2 = Plot(fit_objects = linear_fit2)

p2.plot(fit_info=True)

# Create a contour plot and profiles for the linear fit:
cpf2 = ContoursProfiler(linear_fit2)
cpf2.plot_profiles_contours_matrix(show_grid_for='contours')

# Show the fit results.
plt.show()
```

In [ ]:
# obigen Code hier eingeben:
# --> 


### Kleine Übung: Hinzufügen eines quadratischen Modells

Als kleine Übung soll ein weiteres, quadratisches Modell, $y(x) = ax^2 + b x + c$, hinzugefügt
werden und zusammen mit dem linearen und dem exponentiellen Modell in einer Grafik dargestellt
werden.


In [ ]:
# eingenen Code hier eingeben:
# -->


### 2.7 Relative Unsicherheiten
***

Wir hatten schon gesehen, dass *kafe2* die Angabe von relativen Unsicherheiten 
erlaubt, die wir in diesem Beispiel genauer untersuchen wollen. 

Anpassungen mit relativen Unsicherheiten leiden darunter, dass die Schätzung
der Parameterwerte verzerrt ist.
Denn Messwerte, die zu kleineren Werten fluktuieren, bekommen dadurch kleinere
Unsicherheiten;
für nach oben fluktierende Messwerte sind die Unsicherheiten entsprechend größer.
Wären die zufälligen Fluktuationen genau umgekehrt, würden andere Unsicherheiten zugewiesen.
Richtig wäre es, die relativen Unsicherheiten auf die wahren Werte zu beziehen,
die wir aber nicht kennen.
Stattdessen ermöglicht es die Option ``reference='model'``,
die Unsicherheiten auf den Modellwert zu beziehen - immer noch nicht völlig korrekt,
aber deutlich besser.

Der Effekt lässt sich illustrieren, wenn das Beispiel der linearen Regeression aus
2.1 wiederholt wird,
aber dieses Mal die relativen Unsicherheiten auf die Modellwerte bezogen werden.
Die Unsicherheiten werden nun statt über eine Methode des Datencontainers mittels
der Methode ``add_error()`` des Fit-Objekts angegeben;
die entsprchende Stelle im Code ist mit `-->`  markiert.
Im Code unten wird das ursprüngliche Ergebnis aus 2.1 verwendet und ebenfalls in der
Ausgabegrafik angezeigt, um den Unterschied darzustellen.
Auch Profil-Likelihood und Konturen werden angezeigt, um die Nichlinearität zu demonstrieren.

```    
data1_4 = XYContainer(x_data=x, y_data=y)
data1_4.add_error(axis='x', err_val = 0.3)
data1_4.label = 'Daten, modellbezogene rel. Unsicherheiten'

# Create Fit:
linear_fit4 = Fit(data1_4, model_function=linear_model)
# --> Relative uncertainties with reference to model specified here:
linear_fit4.add_error(axis='y', err_val=0.15, relative=True, reference='model')

# Optional: Assign LaTeX strings to parameters and model functions.
linear_fit4.assign_parameter_latex_names(x='x', a='a', b='b')
linear_fit4.assign_model_function_latex_expression("{a}{x} + {b}")

# Optional: Assign LaTeX strings to parameters and model functions.
linear_fit4.assign_parameter_latex_names(a='a', b='b')
linear_fit4.assign_model_function_latex_expression("{a}{x} + {b}")
linear_fit4.model_label = 'linear m. rel. Unsicherheiten'

# Perform the fit:
linear_fit4.do_fit()

# Optional: print report.
#linear_fit4.report(asymmetric_parameter_errors=True)

# Optional: Create a plot of the fit results using Plot.
p4 = Plot([linear_fit, linear_fit4])
# Assign colors to data ...
p4.customize('data', 'marker', [(0, 'o'), (1,'o')])
p4.customize('data', 'markersize', [(0, 5), (1, 5)])
p4.customize('data', 'color', [(0, 'grey'), (1,'red')]) # note: although 2nd label is suppressed
p4.customize('data', 'ecolor', [(0, 'grey'), (1, 'red')]) # note: although 2nd label is suppressed
# ... and model:
p4.customize('model_line', 'color', [(0, 'mistyrose'),(1, 'orange')])
p4.customize('model_error_band', 'label', [(0, r'$\pm 1 \sigma$'),(1, r'$\pm 1 \sigma$')])
p4.customize('model_error_band', 'color', [(0, 'mistyrose'),(1, 'orange')])

p4.plot(asymmetric_parameter_errors=True)

# Create contour plot and profiles for the linear fit
cpf4 = ContoursProfiler(linear_fit4)
cpf4.plot_profiles_contours_matrix(show_grid_for='contours')

# Show the fit results.
plt.show()   
```   

In [ ]:
# obigen Code hier einfügen:
# -->


Im Vergleich zum Ergebnis aus 2.1 ändern sich die Zentralwerte und die Unschicherheiten werden 
asymmetrisch. Einen großen Effekt haben die beiden Datenpunkte ganz rechts, die nun bezogen auf das Modell kleinere Unsicherheiten und damit einen größeren Einfluss auf die Anpassung bekommen. Als Konsequenz liegt die angepasste Gerade oberhalb von der aus Beispiel 2.1.

---
## 3. Besonderheiten komplexer (nichtlinearer) Modelle
---

Als weiteres Beispiel für eine nicht-lineare Anpassung dient eine gedämpfte Schwingung eines
Fadenpendels.
Die zugehörigen Messdaten sind in der folgenden Code-Zelle enthalten:
```
# The data:
t = [ ... ]
t_errors = 0.05

a = [ ... ]
a_errors = 0.05
```

In [ ]:
# The data:
t = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0,
     10.5, 11.0, 11.5, 12.0, 12.5, 13.0, 13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0,
     19.5, 20.0, 20.5, 21.0, 21.5,22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 25.5, 26.0, 26.5, 27.0, 27.5, 28.0,
     28.5, 29.0, 29.5, 30.0, 30.5, 31.0, 31.5, 32.0, 32.5, 33.0, 33.5, 34.0, 34.5, 35.0, 35.5, 36.0, 36.5, 37.0,
     37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0, 43.5, 44.0, 44.5, 45.0, 45.5, 46.0,
     46.5, 47.0, 47.5, 48.0, 48.5, 49.0, 49.5, 50.0, 50.5, 51.0, 51.5, 52.0,52.5, 53.0, 53.5, 54.0, 54.5, 55.0,
     55.5, 56.0, 56.5, 57.0, 57.5, 58.0, 58.5, 59.0, 59.5, 60.0]
t_errors = 0.05

a = [ 6.06,  5.17,  3.29,  0.64, -2.26, -4.56, -5.74, -5.58, -4.12, -1.62,
      1.11,  3.56,  5.12,  5.43,  4.41,  2.53, -0.18, -2.78, -4.65, -5.5,
     -5.04, -3.25, -0.75,  1.79,  3.88,  5.31,  5.2,   3.92,  1.74, -0.85,
     -3.13, -4.71, -5.06, -4.26, -2.48, -0.13,  2.19,  4.07,  4.9,   4.64,
      3.16,  1.17, -1.54, -3.26, -4.59, -4.64, -3.69, -1.83,  0.38,  2.76,
      4.16,  4.58,  4.13,  2.45,  0.28, -1.8,  -3.53, -4.43, -4.31, -3.03,
     -1.05,  1.06,  2.79,  3.97,  4.4,   3.37,  1.92, -0.14, -2.29, -3.7,
     -4.28, -3.84, -2.44, -0.59,  1.27,  3.11,  3.9,   4.02,  2.85,  1.21,
     -0.64, -2.51, -3.41, -3.84, -3.34, -1.75, -0.17,  1.85,  3.23,  3.72,
      3.4,   2.54,  0.67, -1.13, -2.8,  -3.77, -3.65, -2.89, -1.43,  0.42,
      2.2,   3.26,  3.42,  3.25,  1.88,  0.33, -1.35, -3.02, -3.41, -3.32,
     -2.2,  -0.77,  0.92,  2.44,  3.31,  3.44,  2.77,  1.25, -0.13, -1.69, -2.78 ]
a_errors = 0.05

Die Amplitude in Abhängigkeit von der Zeit ist durch folgende Modellfunktion gegeben:
```
# Model function for a pendulum as a one-dimensional,
#     damped harmonic oscillator with zero initial speed:
# x = time, y_0 = initial_amplitude, l = length of the string,
# r = radius of the steel ball, g = gravitational acceleration, c = damping coefficient.
def damped_harmonic_oscillator(s, a0, l, r, g, c):
  # Effective length of the pendulum = length of the string + radius of the steel ball:
  l_total = l + r
  omega_0 = np.sqrt(g / l_total) # Phase speed of an undamped pendulum.
  omega_d = np.sqrt(omega_0 ** 2 - c ** 2) # Phase speed of a damped pendulum.
  return a0 * np.exp(-c * s) * (np.cos(omega_d * s) + c / omega_d * np.sin(omega_d * s))
```

Daten-Container und Fit-Objekt werden wie üblich erzeugt:
```
# Create data container:
data3 = XYContainer(t, a)
data3.add_error(axis='x', err_val=t_errors)
data3.add_error(axis='y', err_val=a_errors)
data3.axis_labels = ('Time t (s)', 'Amplitude A (°)') 

# Create fit object from data and model function:
fit = Fit(data3, damped_harmonic_oscillator)
```

Das Modell enthält eine Anzahl an Parametern, die durch "Hilfsmessungen" festgelegt sind. 
```
# Relevant physical magnitudes and their uncertainties:
lm, delta_lm = 10.000, 0.002  # length of the string, l = 10.0 +- 0.002 m
rm, delta_rm = 0.052, 0.001  # radius of the steel ball, r = 0.052 +- 0.001 m
# Amplitude of the steel ball at x=0 in degrees, a0m = 6 +- 1% degrees:
a0m, delta_a0m = 6.0, 0.01  # Note that the uncertainty on a0m is relative to a0m.
```

In der Anpassung wird dies berücksichtigt, indem die entsprechenden Parameter sowohl als
Parameter der Anpassung als auch als zusätzliche Datenpunkte berücksichtigt werden.
In kafe2 werden solche durch Messungen eingeschränkte Parameter mit Hifle der Methode
*Fit.add_parameter_constraint()* berücksichtigt und deren Unsicherheiten in das Ergebnis der
Anpassung propagiert:
```
# Constrain model parameters to measurements:
fit.add_parameter_constraint(name='l', value=lm, uncertainty=delta_lm)
fit.add_parameter_constraint(name='r', value=rm, uncertainty=delta_rm)
fit.add_parameter_constraint(name='a0', value=a0m, uncertainty=delta_a0m, relative=True)
```
Als Alternative könnte man die Parameter mit der Methode *Fit.fix_parameter()* auf feste Werte
fixieren;
die Unsicherheiten auf das Endergebnis der Anpassung müssten dann allerdings mit Hilfe der
klassischen Fehlerfortpflanzung berechnet werden.

Als weitere Besonderheit bei nichtlinearen Anpassungen ist zu beachten, dass es häufig
Nebenminima der Kostenfunktion gibt - die Konvergenz zum globalen Minimum kann also nicht
garantiert werden.
Es ist daher notwendig, "vernünftige" Start-Parameter für die Anpassung zu wählen.
Dies geschieht mit Hilfe der Funktion *Fit.set_parameter_values()*:
```
g_initial = 9.81  # Initial guess for g.
fit.set_parameter_values(g=g_initial, a0=a0m, l=lm, r=rm)
```
Wenn die Startwerte gänzlich unbekannt sind, sollten Werte in einem großen Bereich ausprobiert
werden, um zu überprüfen, ob die Anpassungen jeweils zum gleichen Minimum konvergieren. 

Ein weiteres Mittel zur Verbesserung der Konvergenz liegt in der Beschränkung der Parameter
auf "vernünftige" Intervalle.
Die Parameter *a0*, *l*, und *r* sind zum Beispiel per Definition positiv, können während
der Anpassung jedoch auch negative Werte annehmen.
Der folgende Code beschränkt die erwähnten Parameter auf positive Werte:
```
fit.limit_parameter("a0", lower=1e-6)
fit.limit_parameter("l", lower=1e-6)
fit.limit_parameter("r", lower=1e-6)
```
Aus technischen Gründen können Parameter nur auf geschlossene Intervalle beschränkt werden.
Als untere Grenze wird hier deshalb ein kleiner Wert nahe null angegeben.
Da kein oberer Wert angegeben wird, sind die Parameter nur einseitig beschränkt.
Es ist auch möglich, Parameter durch die Angabe von zwei Intervallgrenzen enger einzugrenzen:
```
fit.limit_parameter("g", lower=9.71, upper=9.91)
```
Im obigen Falle beruht die Beschränkung auf der Einschätzung, dass Ergebnisse außerhalb
dieser Grenzen sehr unwahrscheinlich sind.
Es macht auch Sinn, Parameter basierend auf den physikalischen Gegebenheiten des Systems
zu beschränken.
Zum Beispiel liefert die Modellfunktion nur für $c < \frac{g}{l + r}$ reelle Lösungen.
Man kann dies folgendermaßen berücksichtigen:
```
c_max = 0.9 * g_initial / (lm + rm)  # A little lower than our best guess for the limit.
fit.limit_parameter("c", lower=1e-6, upper=c_max)
```

Nach diesen Vorbereitungen kann die Anpassung wie üblich vorgenommen werden. 
Im folgenden Code-Beispiel wird auch gezeigt, wie man über Properties
auf die Fit-Ergebnisse zugreift, fall sie im Programm weiter verarbeitet werden sollen
oder eine spezifische eigene Ausgabe erfolgen soll.
```
# Perform the fit
fit.do_fit()
# Optional: Print out a report on the fit results on the console.
#fit.report(show_data=False, show_model=False, show_fit_results=True)

# Custom printout of results:
print("cost function at minimum: %.4g " % fit.cost_function_value,
    " number of degrees of freedom:", fit.ndf)
print(" --> probability: %.1f%%" % (fit.chi2_probability * 100))
print("parameter names:\n", fit.parameter_names)
np.set_printoptions(precision=5, suppress=False)
print("prameter values:\n", fit.parameter_values)
print("parameter uncertainties:\n",fit.parameter_errors)
np.set_printoptions(precision=3, suppress=True)
print("correlation matrix:\n", fit.parameter_cor_mat )
      
# Optional: plot the fit results.
plot = Plot(fit)
plot.plot(fit_info=True)
plt.show()
```

In [ ]:
# Code hier eingeben
# --> 


---
## 4. Aufbau einer Kovarianz-Matrix aus einzelnen Unsicherheiten
---

Behandelt werden: 
  - der Umgang mit komplexen Unsicherheiten
  
Eine der besonderen Stärken von _kafe2_ ist die Unterstützung von korrelierten Unsicherheiten.
Damit sind Beiträge zur Unsicherheit gemeint, die einige oder alle Werte in gleicher Weise
beeinflussen - z.B. weil sie mit dem gleichen, mit einer systematischen Unsicherheit behafteten
Messgerät aufgezeichnet wurden.
Häufig handelt es sich also um gemeinsame Unsicherheiten von Gruppen von Messwerten.

Zur Angabe von Unsicherheiten dient die Funktion:
> `add_error**( [axis], err_val, name=None, correlation=0, relative=False)`  
  Add an uncertainty source to the data container. Returns an error id which
  uniquely identifies the created error source.  
  **Parameters**  
  • axis (str or int) – 'x'/0 or 'y'/1  
  • err_val (float or iterable of float) – pointwise uncertainty/uncertainties for all data points  
  • name (str or None) – unique name for this uncertainty source. If None, the name
    of the error source will be set to a random alphanumeric string.  
  • correlation (float) – correlation coefficient between any two distinct data points  
  • relative (bool) – if True, err_val will be interpreted as a relative uncertainty  
  **Returns** error name  
  **Return type** str  

Sie gehört zur Container-Klasse, kann aber auch über eine Fit-Klasse aufgerufen werden.
Mit diesem recht einfachen Interface lassen sich sowohl unabhängige Unsicherheiten als auch
gemeinsame absolute oder relative Unsicherheiten von Datenpunkten angeben.
Die angegebenen Unsicherheiten werden in eine Kovarianzmatrix der Datenpunkte umgewandelt.
Bei mehrfachem Aufruf werden die sich ergebenden Kovarianzmatrizen addiert (wie es den Regeln der
elementaren Fehlerfortpflanzung entspricht).

Ein sehr einfach gehaltenes Beispiel soll das illustrieren.
Wir betrachten die Mittelung von vier Werten, die von zwei Gruppen mit unterschiedlichen
Messverfahren durchgeführt wurden.
Jede der beiden Gruppen gibt zwei Messungen an; in der ersten Gruppe gibt es eine absolute, den
beiden Messungen gemeinsame Unsicherheit; die zweite Gruppe gibt eine zwischen ihren beiden
Messungen korrelierte relative - also z.B. durch einen Skalierungsfeher verursachte -
Unsicherheit an.
Den Messungen liegt weiterhin eine gemeinsame (theoretische) Annahme zu Grunde, die zu einer
allen Messungen gemeinsamen, absoluten Unsicherheit führt.

Bei diesem einfachen Problem nutzen wir die einfachste Datenstruktur von *kafe2*,
den _IndexedContainer_, zur Bereitstellung der Daten:  
```
from kafe2 import IndexedContainer
idx_data = IndexedContainer([5.3, 5.2, 4.7, 4.8])  
```
Als Modell wählen wir eine konstante Funktion:
```
# The very simple "model":
def average (a):
  return a
```

Die Unsicherheiten werden dann folgendermaßen angegeben 
                (Anm.: Für _IndexedContainer_ entfällt die Angabe des '_axis_'-Parameters!):
  1. jeder Messung eigene, unabhängige Unsicherheit  
     `err_stat = idx_data.add_error([.2, .2, .2, .2])`
  2. die den ersten beiden Werten gemeinsame Unsicherheit  
     `err_syst12 = idx_data.add_error([.175, .175, 0., 0.], correlation = 1.)`
  3. die den letzten beiden Werten gemeinsame, relative Unsicherheit  
     `err_syst34 = idx_data.add_error([0., 0., .05, .05], correlation = 1., relative=True)`
  4. die allen Werten gemeinsame Unsicherheit   
     `err_syst = idx_data.add_error(0.15, correlation = 1.)`

Wir sollten noch passende Namen für die Daten angeben:
```
idx_data.label = 'Testdaten'
idx_data.axis_labels = [None, 'Messwert (a.u.)']
```

Das Ausführen der Anpassung ist mittlerweile ja gut bekannt:
```
# Set up the fit:
ifit = Fit(idx_data, average)
ifit.model_label = 'Mittelwert'

# Perform the fit:
ifit.do_fit()
```

Die Ergebisse erhält man natürlich mit der _report()_-Funktion, ggf. auch als grafische Darstellung:
```
# Report and plot results:
ifit.report()
p=Plot(ifit)
p.plot()
plt.show()
```

Die Beschriftung der x-Achse ist noch nicht passend - hier sollten nur die Indizes
der Messungen stehen.
Mit ein wenig Hilfe von *matplotlib* lässt sich das erreichen.
Dazu muss auf das *axis*-Objekts der erzeugten Grafik zugegriffen und die entsprechende Anpassung
durchgeführt werden.
Dazu folgenden Code nach der Zeile *p.plot()* vor *plt.show()* eingefügen:
```
# illustrate some a-posteriory fixes to plot layout by accessing the axis object
_ax = p.axes[0]['main']
_ax.set_xticks(range(4)) # Integer axis ticks
```

Wenn ein Problem mehrere Beiträge zur Gesamtunsicherheit enthält, möchte man in der Regel
gerne studieren, welchen Einfluss einzelne Komponenten haben.
Dazu kann man komfortabel mit den Funktionen *disable_error()* und *enable_error()* arbeiten
und entsprechende Anpassungen durchführen:
```
print("disabling common sysytematic error")
idx_data.disable_error(err_syst)
_ifit = Fit(idx_data, average) 
_ifit.do_fit()
_ifit.report()
#      do not forget to switch on again 
idx_data.enable_error(err_syst)
```

In [ ]:
# Code hier eingeben
# -->


---
## 5. Anwendung aus der Praxis: Anpassung einer Breit-Wigner-Resonanz 
---

Behandelt werden: 
  - der Umgang mit komplexen Unsicherheiten
  - das Erzeugen einer ansprechenden grafischen Ausgabe
  - das Studium des Einflusses einzelner Fehlerkomponenten

Typischerweise sind die Unsicherheiten der Messdaten deutlich komplexer als in den bisher
behandelten Beispielen.
Meist sind Unsicherheiten in Ordinate und Abszisse vorhanden, und zusätzlich zu den unabhängigen
Unsicherheiten eines jeden Datenpunktes gibt es allen gemeinsame, korrelierte Unsicherheiten.

Mit der Methode *add_error()* bzw. *add_matrix_error()* können Unsicherheiten auf die '*x*'- und
'*y*'-Daten spezifiziert werden, entweder in Form von unabhängigen bzw. korrelierten, relativen oder
absoluten Unsicherheiten aller oder Gruppen von Messwerten.
Oder durch die Angabe der vollständigen Kovarianz- oder Korrelations-Matrix.
Alle so spezifizierten Unsicherheiten gehen in die globale Kovarianzmatrix für die Anpassung ein.

Als Beispiel betrachten wir Messungen eines Wirkungsquerschnitts als Funktion der Energie in der
Nähe einer Resonanz.
Es handelt sich dabei um kombinierte Messdaten der vier Experimente am Beschleuniger LEP des
CERN, die auf Effekte durch Photon-Abstrahlung korrigiert wurden:
Messungen des hadronischen Wirkungsquerschnitts $\sigma_{e^+e^- \to {\rm hadrons}}$ als Funktion
der Schwerpunktsenergie $E$.
```
## Data:
# Center-of-mass energy E (GeV)
E = [ 88.387, 89.437, 90.223, 91.238, 92.059, 93.004, 93.916 ]  
E_errors = [ 0.005, 0.0015, 0.005, 0.003, 0.005, 0.0015, 0.005 ]
ECor_abs = 0.0017  # correlated absolute errors

# hadronic cross section with photonic corrections applied (nb)
sig = [6.803, 13.965, 26.113, 41.364, 27.535, 13.362, 7.302 ]  
sig_errors = [ 0.036, 0.013, 0.075, 0.010, 0.088, 0.015, 0.045 ]
sigCor_rel = 0.0007 
```

Als Modell verwenden wir eine modifizierte Breit-Wigner-Resonanz mit von der Schwerpunktsenergie
abhängiger Breite ("$s$-dependent width", mit $s = E_{CM}^2$):
```
## Model:
# Breit-Wigner with s-dependent width
def BreitWigner(E, s0 = 41.0, M = 91.2, G = 2.5):
    s = E*E
    Msq = M*M
    Gsq = G*G
    return s0*s*Gsq/((s-Msq)*(s-Msq)+(s*s*Gsq/Msq))
```

Der Daten-Container mit den Unsicherheiten wird wie folgt erzeugt:
```
BWdata= XYContainer(ECM, sig)
# Add independent errors:
error_name_sig = BWdata.add_error(axis='x', name = 'deltaE', err_val = E_errors )   
error_name_E = BWdata.add_error(axis='y', name = 'deltaSig', err_val = sig_errors )
# Add fully correlated, absolute Energy errors:
error_name_ECor = BWdata.add_error(axis='x', name='Ecor',err_val = ECor_abs, correlation = 1.) 
# Add fully correlated, relative cross section errors:
error_name_sigCor = BWdata.add_error(axis='y', name='sigCor', 
                            err_val = sigCor_rel, correlation = 1., relative=True) 
```

Ob es sich um unabhängige oder korrelierte Unsicherheiten handelt, wird durch den Parameter
*correlation* bestimmt;
für unabhängige Unsicherheiten ist er Null, für allen Dateneinträgen gemeinsame Unsicherheiten
ist er Eins.
Werte zwischen 0. und 1. sind ebenfalls zulässig;
allerdings wird in der Praxis die Kovarianzmatreix zur Beschreibung der Gesamtunsicherheit meist
aus unkorrelierten und vollständig korrelierten Komponenten zusammengesetzt.
Die in der Fuktion *add_error* angegebenen Namen erlauben es, später auf die einzelnen
Fehlerkomponenten zuzugreifen.

Anpassung und Ergebnisausgabe folgen der üblichen Vorgehensweise:
```
BWfit = Fit(BWdata, BreitWigner)
BWfit.do_fit()
BWfit.report()
# Optional: plot the fit results
BWplot = Plot(BWfit)
BWplot.plot(fit_info=True)
plt.show()
```

**Verschönerung der grafischen Ausgabe**  
Damit die Art der Daten klar beschrieben ist, sollten noch passende Namen vergeben werden.
Die Zeilen unten müssen dazu vor der Erzeugung des *Fit*-Objekts eingefügt werden.
```
BWdata.label = 'QED-corrected hadronic cross-sections'
BWdata.axis_labels = ('CM Energy (GeV)', '$\sigma_h$ (nb)' )
```
Alternativ können die folgenden Zeilen nach der Erstellung des Fit-Objektes eingefügt werden:
```
BWfit.data_container.label = 'QED-corrected hadronic cross-sections'
BWfit.data_container.axis_labels = ('CM Energy (GeV)', r'$\sigma_h$ (nb)')
```

Es sollte auch noch ein passender Name für das Modell in der Legende für die grafische Ausgabe
gesetzt werden.
Dazu wird die Zeile unten nach der Erzeugung des *Fit*-Objekts eingesetzt:
```
BWfit.model_label = 'Beit-Wigner with s-dependent width'
```

Falls ein schön gesetzter Ausdruck für die Modellfunktion gewünscht wird, können LaTeX-Namen für
das Modell, die Parameter und die Modellfunkton gesetzt werden:
```
# Set LaTeX names for printout in info-box:
BWfit.assign_parameter_latex_names(E='E', s0=r'{\sigma^0}', M=r'{M_Z}', G=r'{\Gamma_Z}')
BWfit.assign_model_function_latex_name(r'\sigma^{\rm ew}_{e^+e^-\to{\rm hadrons}}')
BWfit.assign_model_function_latex_expression(
               r'{s0}\frac{{ {E}^2{G}^2}}{{({E}^2-{M}^2)^2+({E}^4{G}^2/{M}^2)}}')
```

Anmerkung: Die Verdopplung der Klammern "{" und "}" ist notwendig, weil sie in *kafe2*,
ähnlich wie in der Python *format*-Funktion, auch zur Übergabe von Parametern genutzt werden.

Wir haben bereits gesehen, wie man die Bezeichnung für das Band für die Anzeige der
Modellunsicherheit modifizieren kann:
```
BWplot.customize('model_error_band', 'label', [r'$\pm 1\sigma$'])
```

In diesem Beispiel ist allerdings die Modellunsicherheit extrem klein (weit unter 0.1%) und daher
in der Grafik nicht sichtbar.
Unterdrücken kann man die Ausgabe in der Legende mit folgender Angabe:
```
BWplot.customize('model_error_band', 'label', [None])
```

Manchmal wird das Unsicherheitsband von der Line überdeckt; in solchen Fällen solle eine
gestrichelte oder gepunktete Linie für das Modell verwendet werden:
```
BWplot.customize('model_line', 'linestyle', [':'])
```

Nun können noch die Ränder des Plot-Bereiches angepasst werden.
Dies gelingt über die Properties *x_range* und *y_range* der Plot-Klasse:
```
BWplot.x_range = (88, 94)
BWplot.y_range = (0, 45)
```

Da es sich um eine nichtlineare Anpassung handelt, sollten noch Profile-Likelihood 
und Konfidenz-Konturen angezeigt werden.
Die folgende Zeile muss dazu vor *plt.show()* eingefügt werden:
```
ContoursProfiler(BWfit).plot_profiles_contours_matrix(show_grid_for='contours')
``` 
!!! Geduld: die Berechnung der Konturen ist rechenaufwändig und dauert eine gewisse Zeit!


In [ ]:
''' the data for the Breit-Wigner example'''
# Center-of-mass energies E (GeV):
ECM = [ 88.387, 89.437, 90.223, 91.238, 92.059, 93.004, 93.916 ]  
E_errors = [ 0.005, 0.0015, 0.005, 0.003, 0.005, 0.0015, 0.005 ]
ECor_abs = 0.0017  # Correlated absolute errors.

# Hadronic cross sections with photonic corrections applied (nb):
sig = [6.803, 13.965, 26.113, 41.364, 27.535, 13.362, 7.302 ]  
sig_errors = [ 0.036, 0.013, 0.075, 0.010, 0.088, 0.015, 0.045 ]
sigCor_rel = 0.0007 

In [ ]:
# obigen Code hier eingeben 
# --> 


**Studium des Einflusses einzelner Fehlerkomponenten**  
Um den Einfluss einzelner Fehlerkomponenten auf das Ergebnis zu untersuchen, kann man einzelne
Quellen von Unsicherheiten mit der Methode *disable_error()* abschalten und eine neue Anpassung
ausführen, hier gezeigt für die korrelierte Unsicherheit der Schwerpunktsenergien:
```
print('!!!  disabling error component ', error_name_ECor)
BWfit.disable_error(error_name_ECor)
BWfit.do_fit()
BWfit.report(show_data=False, show_model=False)

# do not forget to switch on again !
print('!!!  re-enabling error component ', error_name_ECor)
BWfit.enable_error(error_name_ECor)

#### fallback option with new fit object
#print('!!!  disabling error component ', error_name_ECor)
#BWdata.disable_error(error_name_ECor)
#_fit = Fit(BWdata, BreitWigner)
#_fit.do_fit()
#_fit.report(show_data=False, show_model=False)
#BWdata.enable_error(error_name_ECor)
```

Das Ergebnis ist fast identisch zum vorherigen, lediglich die Unsicherheit der Masse ist nun
kleiner.
Dies war auch so zu erwarten, denn eine korrelierte Änderung aller Energien sollte die Breite
oder Höhe der Resonanz nicht beeinflussen.

Mit der Methode *enable_error(error_name_ECor)* wird die Fehlerquelle wieder aktiviert.

In [ ]:
# hier ausprobieren
# -->


***
## 6. Anpassung an Histogramm-Daten
***

Im Prinzip lässt sich auch die Anpassung einer Verteilungsdichte an eine Häufigkeitsverteilung
als Funktionsanpassung auffassen.
Allerdings gibt es einige Besonderheiten, die berücksichtigt werden müssen:

- Der dem Wert einer Verteilungsdichte (PDF=Particle Density Function) entsprechende
  Funktionswert für ein Bin entspricht dem Integral der PDF über das Bin
- Die Unsicherheit eines Bin-Eintrages ergibt sich aus der Poisson-Verteilung, die nur
  bei sehr großen Zahlen an Einträgen pro Bin durch eine Gauß-Verteilung angenähert werden kann.
   
*kafe2* bietet daher eine spezielle Methode zur Anpassung einer Vereilungsdichte an Histogramme,
die Klassen *HistContainer* zur Abspeicherung der Histogrammdaten und *HistFit* zur
Durchführung der Anpassungen:
```
from kafe2 import HistContainer, HistFit
```

Als Kostenfunktion zur Bewertung der Übereinstimmung der angepassten PDF mit den Bin-Einträgen
in der Häufigkeitsverteilung wird das Doppelte des negativen Logarithmus der Poisson-Likelihood
verwendet, andere Optionen sind konfigurierbar. 

In diesem einfachen Beispiel verwenden wir die Häufigkeitsverteilung von Gauß-verteilten
Zufallszahlen, an die eine Gaußverteilung angepasst wird.
```
def normal_distribution_pdf(x, mu, sigma):
  return np.exp(-0.5 * ((x - mu) / sigma) ** 2) / np.sqrt(2.0 * np.pi * sigma** 2)
```

Die Daten werden zufällig aus der Standardnormalverteilung erzeugt:
```
# create a random dataset of 100 random values, 
#  following a standard normal distribution with mu=0 and sigma=1
data = np.random.normal(loc=0, scale=1, size=100)
```

Der Datencontainer und das Fit-Objekt werden analog zu den früheren Beispielen erstellt:
```
# Create a histogram from the dataset by specifying the bin range and the number of bins.
# Alternatively the bin edges can be set.
histogram = HistContainer(n_bins=10, bin_range=(-5, 5), fill_data=data)

# create the Fit object by specifying a density function
fit = HistFit(data=histogram, model_function=normal_distribution_pdf)
```

Durchführung der Anpassung und Ausgabe der Ergebnisse unterscheiden sich nicht von der
Vorgehensweise bei den früheren Beispielen:
```
# do the fit
fit.do_fit()

# Optional: print a report to the terminal
fit.report()

# Optional: create a plot and show it
phist = Plot(fit)
phist.plot()
plt.show()
```

An dieser Stelle sollten wir noch einmal die Möglichkeiten zur Anpassung der grafischen Ausgabe
anschauen.
Der Plot-Adapter für Histogramme kennt als *plot_type* die Werte *data*, *model* und
*model_density*.
Über den Aufruf von `print(phist.get_keywords(<plot_type>))` können die möglichen Parameter zur
Einstellung ausgebeben werden.
Hier ein Vorschlag für Code zur Anpassung der Grafikausgabe, der vor dem Befehl *phist.plot()*
stehen muss:
```
## reprise: plot customization
#    data
phist.customize('data', 'label', ["random Gaussian data"] ) 
phist.customize('data', 'marker', ['o'])
phist.customize('data', 'markersize', [5])
phist.customize('data', 'color', ['blue']) 
phist.customize('data', 'ecolor', ['blue']) 
#    model
phist.customize('model_density', 'label', ["Gaussian PDF"])
phist.customize('model_density', 'color', ["black"])
phist.customize('model', 'label', ["entries per bin"])
phist.customize('model', 'facecolor', ["lightgrey"])
```

In [ ]:
# hier ausprobieren 
# -->


***
## 7. Likelihood-Anpassungen
***

Wenn nur wenige Messungen vorhanden sind, ist es nicht möglich, eine sinnvolle
Häufigkeitsverteilung zu erhalten, denn eine grobe Einteilung in Bins würde die Messungen
verfälschen, während eine zu feine Einteilung zu Bins mit sehr wenigen oder gar null Einträgen
führen würde.
Das oben schon angewendete Verfahren zur Anpassung einer Verteilungsdichte an eine
Häufigkeitsverteilung ist dann nicht anwendbar.
In solchen Fällen verwendet man eine direkte Anpassung mit Hilfe des
Maximum-Likelihood-Verfahrens and die ungebinnten Daten.
Auch dieses Verfahren ist in *kafe2* implementiert.
Dazu müssen nur die passenden Klassen importiert werden:
```
from kafe2.fit import UnbinnedContainer, UnbinnedFit
```

In diesem Beispiel verwenden wir zur Illustration 160 einzelne Messungen der Lebensdauer von in
einem Detektor gestoppten Myonen aus der kosmischen Strahlung.
Die Häufigkeitsverteilung ist eine Exponentialverteilung über flachem Untergrund:
```
def pdf(t, tau=2.2, fbg=0.1, a=1., b=9.75):
  """
  Probability density function for the decay time of a myon. 
  The pdf is normalized to an integral of one for the interval (a, b).
  :param t: decay time
  :param fbg: background
  :param tau: expected mean of the decay time
  :param a: the minimum decay time which can be measured
  :param b: the maximum decay time which can be measured
  :return: probability for decay time x
  """
  pdf1 = np.exp(-t / tau) / tau / (np.exp(-a / tau) - np.exp(-b / tau))
  pdf2 = 1. / (b - a)
  return (1 - fbg) * pdf1 + fbg * pdf2
```

Zu beachten ist, dass die Häufigkeitsverteilung für alle möglichen Parameterwerte auf Eins
normiert sein muss!

Zur Vorgehensweise bei der Anpassung gibt es nur eine kleine Besonderheit: 
der Untergrundanteil ist auf Grund der geringen Anzahl an Beobachtungen mit einer großen
Unsicherheit behaftet und kann daher bei der Variation im Verlauf des Anpassungsalgorithmus sogar
negativ werden.
Um diesen "unphysikalischen" Bereich des Parameters zu vermeiden, gibt es die Option
`fit.limit_parameter(<name>, lower=<min>, upper=<max> )`.

Alle weiteren Schritte im folgenden Beispielcode sind bereits bekannt:
```
data = UnbinnedContainer(dT) # create the kafe data object
data.label = 'lifetime measurements'
data.axis_labels = ('Myon Life Time ' r'$\tau$' ' (µs)','Density' )

# create the fit object and set the pdf for the fit
LLfit = UnbinnedFit(data=data, model_density_function = pdf)

# assign latex names for model and parameters for nicer display
LLfit.model_label = 'Exponential decay + flat background'
LLfit.assign_parameter_latex_names(t='t', tau=r'\tau', fbg='f', a='a', b='b')
LLfit.assign_model_function_latex_expression("\\frac{{ (1-{fbg}) \, e^{{-{0}/{tau}}}}}"
    "{{{tau}(e^{{-{a}/{tau}}}-e^{{-{b}/{tau}}})}} + \\frac{{ {fbg} }} {{ {b}-{a} }}")

# Fix the parameters a and b ...
a = 1.0
b = 11.5
LLfit.fix_parameter("a", a)
LLfit.fix_parameter("b", b)
# ... and limit parameter fbg
LLfit.limit_parameter("fbg", lower=0., upper=1.)

LLfit.do_fit()  # perform the fit
LLfit.report(asymmetric_parameter_errors=True)

pLL = Plot(LLfit)  # create a plot object
pLL.x_range = [a, b]
pLL.plot(fit_info=True, asymmetric_parameter_errors=True)  # plot the data and the fit
#pLL.axes[0]['main'].set_xlabel('Life time '+r'$\tau$'+' (µs)', size='large')  # overwrite the x-axis label

cpfLL = ContoursProfiler(LLfit, profile_subtract_min=False)  # Optional: create a contours profile
cpfLL.plot_profiles_contours_matrix(parameters=['tau', 'fbg'])  # Optional: plot the contour matrix for tau and fbg

plt.show()  # show the plot(s)
```

Interessant ist die spezielle Form der grafischen Darstellung der Daten, bei der in diesem Fall
jeder Messwert durch einen Strich dargestellt wird.
Die Dichte der Striche pro Längeneinheit entspricht der Verteilungsdichte.

In [ ]:
''' the data for the myon life time example'''
# real data from measurement with a Water Cherenkov detector ("Kamiokanne")
dT = [7.42, 3.773, 5.968, 4.924,  1.468,  4.664,  1.745,  2.144,  3.836,  3.132,
  1.568,  2.352,  2.132,  9.381,  1.484,  1.181,  5.004,  3.06,   4.582,  2.076,
  1.88,   1.337,  3.092,  2.265,  1.208,  2.753,  4.457,  3.499,  8.192,  5.101,
  1.572,  5.152,  4.181,  3.52,   1.344, 10.29,   1.152,  2.348,  2.228,  2.172,
  7.448,  1.108,  4.344,  2.042,  5.088,  1.02,   1.051,  1.987,  1.935,  3.773,
  4.092,  1.628,  1.688,  4.502,  4.687,  6.755,  2.56,   1.208,  2.649,  1.012,
  1.73,   2.164,  1.728,  4.646,  2.916,  1.101,  2.54,   1.02,   1.176,  4.716,
  9.671,  1.692,  9.292, 10.72,   2.164,  2.084,  2.616,  1.584,  5.236,  3.663,
  3.624,  1.051,  1.544,  1.496,  1.883,  1.92,   5.968,  5.89,   2.896,  2.76,
  1.475,  2.644,  3.6,    5.324,  8.361,  3.052,  7.703,  3.83,   1.444,  1.343,
  4.736,  8.7,    6.192,  5.796,  1.4,    3.392,  7.808,  6.344,  1.884,  2.332, 
  1.76,   4.344,  2.988,  7.44,   5.804,  9.5,    9.904,  3.196,  3.012,  6.056, 
  6.328,  9.064,  3.068,  9.352,  1.936,  1.08,   1.984,  1.792,  9.384, 10.15,   
  4.756,  1.52,   3.912,  1.712, 10.57,   5.304,  2.968,  9.632,  7.116, 1.212,
  8.532,  3.000,  4.792,  2.512,  1.352,  2.168,  4.344,  1.316,  1.468, 1.152,
  6.024,  3.272,  4.96,  10.16,   2.14,   2.856, 10.01,   1.232, 2.668, 9.176 ]

In [ ]:
# Likelihood-Anpassung hier ausprobieren
# -->


***
## 8. Multi-Fits:
### simultane Anpassung von Modellfunktionen an verschiedene Datensätze
***

Sehr oft sind die Modelle zu komplex, um alle Parameter in einer Anpassung an ein
einziges Modell zu bestimmen.
Modellparameter sind oftmals das Ergebniss mehrerer Modellanpassungen, oder der
selbe Parameter kommt in verschiedenen Messreihen vor.

Für solche Fälle bietet *kafe2* die Möglichkeit, mehrere Anpassungen unterschiedlicher Modelle
mit gemeinsamen Parametern an verschiedene Datensätze durchzuführen.

Dazu muss zusätzlich das Paket *MultiFit* importiert werden:
```
from kafe2 import MultiFit
```

Wir betrachten als einfaches Beispiel die Bestimmung eines ohmschen Widerstands bei
Zimmertemperatur, der sich bei höherem Stromfluss erwärmt und so seinen Widerstand gemäß seines
Temperaturkoeffizienten ändert.
Zusätzlich zum Strom durch den Widerstand wird daher noch die Temperatur für jeden vorgegebenen
Spannungswert gemessen.
Es müssen also Triplets von Messwerten ausgewertet werden.

Die Temperaturabhängigkeit wird empirisch durch ein einfaches quadratisches Modell beschreiben:
```
# empirical model for T(U): a parabola
def empirical_T_U_model(U, p2=1.0, p1=1.0, p0=0.0):
    # use quadratic model as empirical temperature dependence T(U)
    return p2 * U**2 + p1 * U + p0
```

Der Widerstand als Funktion der Temperatur ist durch den Temperaturkoeffizienten $\alpha$ gegeben
und wird folgendermaßen modelliert:
```
# model of current-voltage dependence I(U) for a heating resistor
def I_U_model(U, R0=1., alph=0.004, p2=1.0, p1=1.0, p0=0.0):
    # use quadratic model as empirical temperature dependence T(U)
    t_ref = 0.
    _delta_t = empirical_T_U_model(U, p2, p1, p0) - t_ref
    # plug the temperature into the model
    return U / (R0 * (1.0 + _delta_t * alph))
```
Das Modell für den Widerstand enthält also in diesm Fall das erste Modell für die Abhängigkeit
der Temperatur von dem durch die angelegte Spannung bestimmten Strom.

Hier die Daten für dieses Beispiel:
```
# the data 
U = [ 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5,   
      6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0, 9.5, 10.0 ] 
I = [ 0.5,  0.89, 1.41, 1.67, 2.3,  2.59, 2.77, 3.57, 3.94,  4.24, 4.73,
      4.87, 5.35, 5.74, 5.77, 6.17, 6.32, 6.83, 6.87, 7.17 ]
T = [ 20.35, 20.65, 22.25, 23.65, 26.25, 27.85, 29.85, 34.25, 37.75, 41.95,
     44.85, 50.05, 54.25, 60.55, 65.05, 69.95, 76.85, 81.55, 85.45, 94.75 ]
sigU, sigI, sigT = 0.2, 0.1, 0.5 # uncertainties
```

Die Fit-Prozedur unterscheidet sich kaum von der bisher vorgestellten Vorgehensweise. 
Zunächst werden die Daten-Container und Anpassungen für die beiden Modelle definiert:
```
# Step 1: construct the singular data containers and fit objects
TU_data = XYContainer(U,T)
TU_data.label = 'Temperaturen'
TU_data.axis_labels = ['Spannung (V)','Temperatur (°C)']
fit_1 = Fit(TU_data, model_function=empirical_T_U_model)
fit_1.model_label = 'Parametrisierung'

IU_data = XYContainer(U,I)
IU_data.label = 'Ströme'
IU_data.axis_labels = ['Spannung (V)','Strom (A)']
fit_2 = Fit(IU_data, model_function=I_U_model)
fit_2.model_label = 'Temperaturabhängiger Leitwert'

```

Dann werden beide Anpassungen zu einem *MultiFit* zusammengefasst.
```
# Step 2: construct a MultiFit object
multi_fit = MultiFit(fit_list=[fit_1, fit_2], minimizer='iminuit')
```
Erst jetzt werden die Unsicherheiten - dieses Mal zu den
Fit-Objekten, hinzugefügt. Dadurch können auch die in beiden
Datensätzen gemeinsamen Unsicherheiten auf der x-Achse berücksichtigt 
werden. 
```
# Step 3: Add errors (to the fit object in this case)
multi_fit.add_error(sigT, 0, axis='y')  # declare errors on T
multi_fit.add_error(sigI, 1, axis='y')  # declare errors on I
multi_fit.add_error(sigU, 'all', axis='x') # shared error on x axis
```

Es folgt noch die Definition aussagekräftiger Namen für die Ausgabe:
```
# (Optional): assign names for models and parameters
multi_fit.assign_parameter_latex_names(
    U='U', p2='p_2', p1='p_1', p0='p_0', R0='R_0', alph=r'\alpha_\mathrm{T}')
multi_fit.assign_model_function_expression('{p2}*{U}^2 + {p1}*{U} + {p0}', fit_index=0)
multi_fit.assign_model_function_latex_expression(r'{p2}\,{U}^2 + {p1}\,{U} + {p0}', fit_index=0)
multi_fit.assign_model_function_expression('{U} / ({R0} * (1 + ({p2}*{U}^2 + {p1}*{U} + {p0}) * {alph}))', fit_index=1)
multi_fit.assign_model_function_latex_expression(r'\frac{{{U}}}{{{R0} \cdot (1 + ({p2}{U}^2 + {p1}{U} + {p0}) \cdot {alph})}}', fit_index=1)
```

Der Rest läuft dann genau so wie schon oft gezeigt:
```
# Step 4: do the fit
multi_fit.do_fit()

# (Optional): print the results
multi_fit.report()

# (Optional): plot the results
plot = Plot(multi_fit, separate_figures=True)
plot.customize('data', 'marker', ['.','.'])
plot.customize('data', 'markersize', [6,6])

plot.plot()

plt.show()
```


In [ ]:
# eigenen Code hier eingeben
# -->

